<a href="https://colab.research.google.com/github/sethorus30/API_Project/blob/master/Capstone_Soup_Working_Code_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Author:  Adam Ezzat
# This application is for Web Scrapping data from the kbb.com web site. It correlates the scrapped 
# data with a dataset from Kaggle representing data about used cars from Pakistan.
# The analysis investigates what features cars had to reduce the depreciasion rate of a car value.
# The application consists of 4 modules (step-1, step-2, step-3, and step-4) shown below.
# In addition, below step-1 is the output of running the code on the relevant car models from kbb.com.


# Step-1: This file is to generate Single Maker data frame given the Maker URL

from     bs4 import  BeautifulSoup as soup
import   requests
import   pandas as pd
from     urllib.request  import  urlopen
import   xml.etree.ElementTree  as ET
import   copy

# Create list for Maker’s home page URL
makers_List = ['https://www.kbb.com/acura/' , 'https://www.kbb.com/alfaromeo/' , 'https://www.kbb.com/astonmartin/' , 'https://www.kbb.com/audi/' , 'https://www.kbb.com/bentley/',  'https://www.kbb.com/bmw/' , 'https://www.kbb.com/buick/' , 'https://www.kbb.com/cadillac/' , 'https://www.kbb.com/chevrolet/' , 'https://www.kbb.com/chrysler/' , 'https://www.kbb.com/dodge/' , 'https://www.kbb.com/ferrari/' , 'https://www.kbb.com/fiat/' , 'https://www.kbb.com/ford/' , 'https://www.kbb.com/freightliner/' , 'https://www.kbb.com/genesis/' , 'https://www.kbb.com/gmc/' , 'https://www.kbb.com/honda/' , 'https://www.kbb.com/hyundai/' , 'https://www.kbb.com/infiniti/' , 'https://www.kbb.com/jaguar/' , 'https://www.kbb.com/jeep/' , 'https://www.kbb.com/kia/' , 'https://www.kbb.com/lamborghini/' , 'https://www.kbb.com/landrover/',  'https://www.kbb.com/lexus/' , 'https://www.kbb.com/lincoln/' , 'https://www.kbb.com/maserati/' , 'https://www.kbb.com/mazda/' , 'https://www.kbb.com/mclaren/' , 'https://www.kbb.com/mercedesbenz/' , 'https://www.kbb.com/mini/' , 'https://www.kbb.com/mitsubishi/' , 'https://www.kbb.com/nissan/' , 'https://www.kbb.com/porsche/', 'https://www.kbb.com/ram/' , 'https://www.kbb.com/rivian/' , 'https://www.kbb.com/rollsroyce/' , 'https://www.kbb.com/smart/' , 'https://www.kbb.com/subaru/' , 'https://www.kbb.com/tesla/' , 'https://www.kbb.com/toyota/' , 'https://www.kbb.com/volkswagen/' , 'https://www.kbb.com/volvo/']

# Create list for annual inflation rate in the USA  (2020 - 1964)
# URL: https://www.usinflationcalculator.com/inflation/historical-inflation-rates/  
annual_inflation_rate_List = [0 , 0.018, 0.024, 0.021, 0.013, 0.001, 0.016, 0.015, 0.021, 0.032, 0.016, -0.004, 0.038, 0.028, 0.032, 0.034, 0.027, 0.023, 0.016, 0.028, 0.034, 0.022, 0.016, 0.023, 0.03, 0.028, 0.026, 0.03, 0.042, 0.054, 0.048, 0.041, 0.036, 0.019, 0.036, 0.043, 0.032, 0.062, 0.103, 0.135, 0.113, 0.076, 0.065, 0.058, 0.091, 0.11, 0.062, 0.032, 0.044, 0.057, 0.055, 0.042, 0.031, 0.029, 0.016, 0.013, 0.013]

mListSize = len(makers_List)
print ('Maker List size = ' , mListSize)
print("\n\n")
prop = {}
tmp = {}
tmp_dct_p = {}
tmp_dct = {}
links = []
Maker = ''
first_year = ''
i = 0
m = []
values = []
model_dct_list = []                 # list of all models for a given maker
tmp_dct_list = []       # list of dictionaries structure used by called functions.
model_years_dct_list = []  # holds detailed model info (model, first-year, price, model_url>
model_final_dct_list = []      # final including all years for given model

# input:  maker_url
# output: {Maker : maker, Model : v1,  First Year : v2,  New_Price : v3,  url : link}, {}….] }
def  get_maker_models(maker_url) : 
    r3 = requests.get(maker_url)
    #print("maker_url: ", maker_url)
    soup1 = soup( r3.content, 'html.parser')

    # find iterative top level TAG for each maker/model
    link_as = soup1.find('div', {"class" : "make-list-wrapper"}).find_all('div', {"class" : "results-row js-results-row"})
    
    i = 0
    # y is the top-level TAG for 1st year models
    # from that top level get: links, maker, model, first year for all 
    # top-year Models for that Maker’s new Cars
    for   y  in   link_as :       
        x = y.find('a', href=True)
        link = "https://www.kbb.com" + x['href']   # x['href'] has '/Maker/Model'
        if link not in links:
            links.append(link)
            maker, model = (x['href'].strip('/')).split('/')
            Maker = maker[:]
            # get the first year from "2019 Aston Marton DB11"
            ymm = x.find('h3', {"class" : "title-three"}).string 
            print("ymm: ", ymm)            
            m = ymm.split()
            print("9: ", m[0], type(m[0]))                                            # m[0] is 1st year = 2019 is a string
            # first-year = m[0]
            # get the price for the first year
            price = y.find_next('span', {"class" : "title-four"}).text
            print ("price = ", price)
            L1 = ['Maker', 'Model', 'Year', 'New_Price', 'Model-url']
            L2 = [Maker, model, m[0], price, link]
            dct = dict(zip(L1, L2))
            if  i == 0 :
                # copy dct to model_dct_list[0]
                model_dct_list.insert(0, dct.copy())
            else : 
                model_dct_list.append(dct.copy())
            i = i + 1

    print ("\n\n")   
    #print("1: ", model_dct_list) 
    return    model_dct_list                

# Input: model, fyear, fprice, model_url   1st year data for specific maker/model
# Output: [ {model1, year1, price1, model_url1}, {model1, year2, price2, model_url2}, ….]

def   get_maker_model_years_price (maker, model, f_year, f_price, model_url) :
    cur_price = 0
    next_year_price = 0
    r = requests.get(model_url)
    # create soup2 object for the model page for a specific Maker (to search 
    # recursively for <year and Link> )
    soup2 = soup( r.content, 'html.parser')
    # find years and corresponding links for this specific Maker/Model/First-
    # Year/New_Price/Model_url of new Cars
    i = 0
    # convert price (f_price) from string ($xx,xxx) into integer binary representation
    pp = f_price.replace('$', '')
    pp = pp.replace(',', '')
    cur_price = int(pp)                                                       # convert string pp into integer
    next_year_price = cur_price          # integer binary representation
    
    # write/append first year data for a given model into tmp_dct_list[]
    tmp_dct = {}
    tmp_dct_list.clear()
    L1 = ['Maker', 'Model', 'Year', 'New_Price', 'Model-url']
    L2 = [maker, model, f_year, f_price, model_url]
    tmp_dct = dict(zip(L1, L2))
    # add first year for this specific maker/model
    tmp_dct_list.insert(0, tmp_dct.copy())  
    
    # find index in the annual_inflation_rate_list[] for this model
    if  '2020' in f_year :                  # 2020
        i = 1                               # inflation starts from 2019
    elif  '2019'  in  f_year :              # 2019
        i = 2                               # inflation starts from 2018
    elif   '2018'  in f_year :              # 2018
        i = 3                               # inflation starts from 2017
    else :
        i = -1                               # undefined
        
    last_year = f_year[ : ]
    n = 0                                                                     # skip first year as I have it already
    # remaining years below
    for  y  in  soup2.find_all("li", {"data-analytics" : True}) : 
        if  n == 0 :             # skip first year as I have it already
            n = n + 1
            continue
        mm_year = y.find_next('a', href=True)# mm_year = “/acura/ilx/2019”
        link = "https://www.kbb.com" + mm_year['href']
        year = y.find_next('a', href=True).contents    # year = “2019” 
        print("type(year): ", type(year))
        print("year: ", year)
        yr = str(year[0])                               # string
        print ("f_year, p_year", f_year, yr)
        if yr == last_year or yr == "" :
            i = i + 1
            continue
        last_year = yr[ : ]
        # estimated cur_price (binary) = 
        #              int(next_year_price  / (1 + annual_inflation_rate_List[i]) )
        # cur_price is integer binary representation
        cur_price = int(next_year_price / (1 + annual_inflation_rate_List[i]) ) 
        next_year_price = cur_price
        p1 = str(cur_price)                # convert binary into string
        l = len(p1) - 3                    # l = (string length – 3)
        p2 = p1[:l] + ',' +  p1[l:]
        p = "$" + p2         # p is current price in string with right format
        #p = "N/A"
        if (len(tmp_dct)) :
            tmp_dct.clear()
        # Needs to adjust the URL by appending year
        m_url = model_url + yr
        L1 = ['Maker', 'Model', 'Year', 'New_Price', 'Model-url']
        L2 = [maker, model, year, p, m_url]
        tmp_dct = dict(zip(L1, L2))         
        tmp_dct_list.append (tmp_dct.copy() )
        i = i + 1                                                                  # index into annual inflation rate array
    return  tmp_dct_list     # [ {<maker: <model, year, price, model_url} ]

# input: maker/model/<year : URL> and output: maker/model/years/price/{properties dictionary}
def get_maker_model_year_price_properties(maker, model, year, price, model_year_url) :
    prop.clear()                           # clear the dictionary
    technology = []
    entertainment = []
    #tmp_dct = {}
    i = 0    
    r = requests.get(model_year_url)
    # create soup3 object for the top level HTML page (to search recursively
    # for makers, models and Links)
    soup3 = soup( r.content, 'html.parser')  
    # print(soup3.find('div', {"class" : "css-1wbw7cx epomrkc1"}).find('div', {"class":"stars"}).text)
    # (1-2) find out MPG and HorsePower
    mpg = False
    hp = False
    print("get_maker_model_year_price_properties - year: ", year)

    for p  in soup3.find('div', {"class" : "css-1wbw7cx epomrkc1"}).find_all('div', {"class" : "stars"}):
        parent = str(p.find_parent('div').text).strip()
        #print("parent: ", parent)
        if  "Combined Fuel Economy" in   parent :
            #prop['MPG'] = p.text
            M_P_G = p.text
            mpg = True
        elif  "Horsepower"  in   parent :
            #prop['HP'] = p.text
            H_P = p.text
            hp = True
        else :
            break
    if mpg == False :
        #prop['MPG'] = "N/A"
        M_P_G = "N/A"
    if hp == False :
          #prop['HP'] = "N/A"
          H_P = "N/A"

    # (3-4) find out Expert Rating and Consumer rating
    i = 0
    er = False
    cr = False
    for p in soup3.find_all('div', {"class":"css-x9skgx-NumericRating e149jidm1"}) :
        i = i + 1
        if i == 1 :
            if  p.text :
                value = p.text.replace('\xa0', '')
                #prop['Expert-Rating'] = value
                E_R = value
                er = True
            else :
                prop['Expert-Rating'] = "N/A"
                E_R = "N/A"
        elif (i == 2 ): 
            if p.text :
                value = p.text.replace('\xa0', '')
                #prop['Consumer-Rating'] =  value
                C_R = value
                cr = True
            else :
                prop['Consumer-Rating'] =  "N/A"
                C_R = "N/A"
        else :
            break
    if er == False :
      E_R = "N/A"
    if cr == False :
      C_R = False

    # (5) Avalon safety rating
    sr = False
    safety = soup3.find('div', {"class":"css-b7zitt e1pswftg4"})
    if safety :
        S_R = safety.contents[0]
        sr = True
    else :
        S_R = "N/A"
    
    if sr == False :
      S_R = "N/A"
   
    # (6) Avalon Technology
    for  p   in soup3.find('ul', {"class":"css-p619jy-StyledList-List"}).find_all('span', {"class" : "css-lml4x1-ListContent e1jmyeyl2"}) :
        technology.append(p.text)

    #prop['Technology'] = technology

    # (7) Avalon Entertainment: skip Technology
    for  p   in   soup3.find_all('ul', {"class":"css-p619jy-StyledList-List"})[1] :
        for  e   in  p.find_all('span', {"class" : "css-lml4x1-ListContent e1jmyeyl2"}) :
            entertainment.append(e.text)
 
    #prop['Entertainment'] = entertainment
    #L1 = ['Maker', 'Model', 'Year', 'New_Price', 'property']
    #L2 = [maker, model, year, price, prop]
    L1 = ['Maker', 'Model', 'Year', 'New_Price', 'MPG', 'HP', 'Expert-Rating', 'Consumer-Rating', 'Safety-Rating', 'Technology', 'Bluetooth Wireless Technology', 'Cruise Control', 'Hands Free Phone', 'Navigation System', 'Remote Keyless Entry', 'Audible Text Messaging', 'Internet Access', 'Proximity Sensing Keyless Entry', 'Push-Button Engine Start', 'Real-Time Traffic Information', 'Smartphone Interface', 'Touch Screen Monitor', 'USB Port', 'Voice Recognition System', 'Remote Engine Start', 'Entertainment', 'iPod Connector', 'MP3 Player', 'Premium Radio', 'Satellite Radio', 'Bluetooth Streaming Audio', 'CD Player', 'iPod Interface']
    L2 = [maker, model, year, price, M_P_G, H_P, E_R, C_R, S_R, technology, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, entertainment, 0, 0, 0, 0, 0, 0, 0]
    tmp_dct_p = dict(zip(L1, L2))
    print("tmp_dct_p; ", tmp_dct_p)
    return  tmp_dct_p              # return prop dictionary <key : value>

# start of main program
# --------------------------------------------------------------
# select one Maker URL from the Maker list.   For the selected Maker get all
# corresponding models
# read list of dictionaries as the value of maker_dct[Maker].  Each is a URL 
# for Maker page
# read one maker page Url from the makers_list[]
model_dct_list = get_maker_models(makers_List[41])
print("1: list of [model, fyear, fprice, model_url]: " , model_dct_list)
print("\n\n")

# For all models: for each model first year , get the rest of the years data for that model
model_years_dct_list.clear()      # clear model_final_dct[] before populating

# For each model: pass 1st year data and gets back the data for all years for that model
length  = len (model_dct_list)
print("model_dct_list length = ", length)
for    i    in  range (length) :
    tmp_dct.clear()
    tmp_dct = model_dct_list[i].copy()
    maker = tmp_dct.get('Maker')
    model = tmp_dct.get('Model')
    year = tmp_dct.get('Year')
    if i != 0 :
      print ("year = ", year)			# [‘2018’]
      year.replace('[', '')
      year.replace(']', '')
      print ("year = ", year)

    fprice = tmp_dct.get('New_Price')
    model_url = tmp_dct.get('Model-url')
    
    if i == 0 :
        model_years_dct_list.insert(0, get_maker_model_years_price(maker, model, year, fprice, model_url).copy()) 
    else :
        model_years_dct_list.append(get_maker_model_years_price(maker, model, year, fprice, model_url).copy())
            
print("2:  model_years_dct_list[] = : ",  model_years_dct_list)
print("2: length of model_years_dct_list = ", len(model_years_dct_list))
print("done with (2) \n\n")

# for each models/years get that model/year properties into model_final_dct_list[]
# clear model_final_dct[] before populating
model_final_dct_list.clear()                       
length = len (model_years_dct_list) 
print("8: ", length)
sum = 0
ll = len(model_years_dct_list)
for i in range (ll) :
    sum  +=  len(model_years_dct_list[i])
print ("sum = ", sum)
for i  in range (length):
    k = len(model_years_dct_list[i])
    tmp_dct.clear()
    tmp_dct = model_years_dct_list[i].copy()
    for j in range (k) :
        maker = tmp_dct[j].get('Maker')
        model = tmp_dct[j].get('Model')
        year = tmp_dct[j].get('Year')                # List format ['2017']
        #print("year type: ", type(year))
        #print("year: ", year)
        s_year = str(year[0])
        s_year.replace('[', '')
        s_year.replace(']', '')
        #print("s_year: ", s_year)
        price = tmp_dct[j].get('New_Price')
        # get the model URL for the most recent year
        # convert to binary -> subtract one -> convert back to string
        model_year_url = str(tmp_dct[j].get('Model-url'))
        #print("model_year_url: ", model_year_url)
        
        #if  i == 0 and  j == 0 :
        if  j == 0 :
            #print("j = 0 : ", maker, model, year, price, model_year_url)
            #model_final_dct_list.insert(0, copy.deepcopy(get_maker_model_year_price_properties(maker, model, year, price, model_year_url)))
            #model_final_dct_list.insert(0, get_maker_model_year_price_properties(maker, model, year, price, model_year_url).copy())
            model_final_dct_list.append(get_maker_model_year_price_properties(maker, model, year, price, model_year_url))
            print("j = 0: model_final_dct_list length = ", len(model_final_dct_list))
            print("j = 0: ", model_final_dct_list[0], "\n")
        else :
            #print("j != 0 ", maker, model, s_year, price, model_year_url)
            #model_final_dct_list.append(copy.deepcopy(get_maker_model_year_price_properties(maker, model, s_year, price, model_year_url)))
            model_final_dct_list.append(get_maker_model_year_price_properties(maker, model, s_year, price, model_year_url))
            #model_final_dct_list.append(dict(get_maker_model_year_price_properties(maker, model, s_year, price, model_year_url)))
            #print("j != 0: model_final_dct_list length = ", len(model_final_dct_list))
            #print("j != 0: index = 0 ", model_final_dct_list[0], "\n")
            #print("j != 0: index = last entry ", model_final_dct_list[-1], "\n")

print ("3: size of model_final_dct_list = ", len(model_final_dct_list))
print("3: model_final_dct_list[0]", model_final_dct_list[0])
print("3: model_final_dct_list[5]", model_final_dct_list[5])
#print("3: model_final_dct_list[] = : ", model_final_dct_list)
print("\n\n")
# build the data frame for a single Maker
Maker42_df = pd.DataFrame (model_final_dct_list)
print("4: Done\n\n")

# Save Maker1_df.csv on the computer under Adam Home page\Springboard\Makers-DFs folder
#Maker1_df.to_csv (r'C:\Users\Default\Desktop\Maker1_DataFrame.csv', index=False)
Maker42_df.to_csv (r'F:\Users\AdamEzzat\Springboard\Makers-DFs\Maker42_df.csv')

# Read the csv file back as Data Frame in your runtime Cell
# Maker1_df = pd.read_csdone v("C:\Users\AdamEzzat\Springboard\Makers-DFs\Maker1_df.csv")
print("5: CSV is done\n\n")


Maker List size =  44



ymm:  
				2020 Toyota Avalon
			
9:  2020 <class 'str'>
price =  $36,830
ymm:  
				2020 Toyota Avalon Hybrid
			
9:  2020 <class 'str'>
price =  $37,955
ymm:  
				2020 Toyota Camry
			
9:  2020 <class 'str'>
price =  $25,380
ymm:  
				2020 Toyota Camry Hybrid
			
9:  2020 <class 'str'>
price =  $29,385
ymm:  
				2020 Toyota Corolla
			
9:  2020 <class 'str'>
price =  $20,555
ymm:  
				2020 Toyota Corolla Hybrid
			
9:  2020 <class 'str'>
price =  $24,055
ymm:  
				2019 Toyota Mirai
			
9:  2019 <class 'str'>
price =  $59,455
ymm:  
				2020 Toyota Prius
			
9:  2020 <class 'str'>
price =  $25,280
ymm:  
				2020 Toyota Prius Prime
			
9:  2020 <class 'str'>
price =  $28,855
ymm:  
				2020 Toyota Yaris
			
9:  2020 <class 'str'>
price =  $16,605
ymm:  
				2020 Toyota 4Runner
			
9:  2020 <class 'str'>
price =  $37,240
ymm:  
				2020 Toyota C-HR
			
9:  2020 <class 'str'>
price =  $22,415
ymm:  
				2020 Toyota Highlander
			
9:  2020 <class 'str'>
pric

In [ ]:
import pandas as pd
Global_df = pd.read_csv('/content/Global_df .csv')

In [ ]:
import pandas as pd
Pakistan_df = pd.read_csv('/content/Pakistan_df.csv',  encoding='cp1252')

In [ ]:
# Step-2: This program is to concatenate set of Makers data frame into Global_df
# and store it as CSV file on local disk (Global_df.csv

from     bs4 import  BeautifulSoup as soup
import   requests
import   pandas as pd
from     urllib.request  import  urlopen
from     google.colab  import files

# Bring CSV files (e.g., Maker1_df.csv) into memory as Data Frame objects (e.g., Maker1_df):
Maker1_df = pd.read_csv(r'/content/F:\AdamEzzat\Springboard\Makers-DFs\Maker1_df.csv')
Maker4_df = pd.read_csv(r'/content/F:\AdamEzzat\Springboard\Makers-DFs\Maker4_df.csv')
Maker6_df = pd.read_csv(r'/content/F:\AdamEzzat\Springboard\Makers-DFs\Maker6_df.csv')
Maker9_df = pd.read_csv(r'/content/F:\AdamEzzat\Springboard\Makers-DFs\Maker9_df.csv')
Maker18_df = pd.read_csv(r'/content/F:\AdamEzzat\Springboard\Makers-DFs\Maker18_df.csv')
Maker19_df = pd.read_csv(r'/content/F:\AdamEzzat\Springboard\Makers-DFs\Maker19_df.csv')
Maker23_df = pd.read_csv(r'/content/F:\AdamEzzat\Springboard\Makers-DFs\Maker23_df.csv')
Maker26_df = pd.read_csv(r'/content/F:\AdamEzzat\Springboard\Makers-DFs\Maker26_df.csv')
Maker29_df = pd.read_csv(r'/content/F:\AdamEzzat\Springboard\Makers-DFs\Maker29_df.csv')
Maker33_df = pd.read_csv(r'/content/F:\AdamEzzat\Springboard\Makers-DFs\Maker33_df.csv')
Maker34_df = pd.read_csv(r'/content/F:\AdamEzzat\Springboard\Makers-DFs\Maker34_df.csv')
Maker42_df = pd.read_csv(r'/content/F:\AdamEzzat\Springboard\Makers-DFs\Maker42_df.csv')


# Concatenate these Data Frame files into one Data Frame
df = [ Maker1_df, Maker4_df, Maker6_df, Maker9_df, Maker18_df, Maker19_df, Maker23_df, Maker26_df, Maker29_df, Maker33_df, Maker34_df, Maker42_df ] 

list = []

# Store this single Data Frame file as CSV file on local disk
Global_df = pd.concat(df)
print("0: Global_df: ", Global_df)
print("0: type(Global_df): ", type(Global_df))

# Write CSV files (e.g., Global_df.csv) from the data objects (e.g., Global_df) with expanded 
# Technology & Entertainment:
# We have already the Pakistan_df.csv
#Global_df.to_csv (r'F:\Users\AdamEzzat\Springboard\BeautifulSoup\Makers-DFs\Global_df.csv')
Global_df.to_csv('Global_df.csv')
files.download('Global_df.csv')   # now on Colab LHS download the CSV file to disk


In [ ]:
# Step-3: This program is to expand & initialize both the Technology and Entertainment 
# columns in the Global_df data frame and save it as CSV file on local disk 
# (Global_df.csv)

from     bs4 import  BeautifulSoup as soup
import   requests
import   pandas as pd
from     urllib.request  import  urlopen
import   copy
import   ast


# read Global_df.csv into Colab if needed
#Global_df = pd.read_csv(r'/content/Global_df.csv')
Global_df = pd.read_csv('/content/Global_df.csv')

Global_df

# Adam’s code to initialize the expanded Technology (15 columns) in 
# Global_df data frame.  Each row in Global_df will support a subset of
# the 15 columns for Technology (set to 1) and the remaining columns 
# (are set to 0)

# create a list for all 15 keys (column name) for the Technology dictionary
tech = ['Bluetooth Wireless Technology', 'Cruise Control', 'Hands Free Phone', 'Navigation System', 'Remote Keyless Entry', 'Audible Text Messaging', 'Internet Access', 'Proximity Sensing Keyless Entry', 'Push-Button Engine Start', 'Real-Time Traffic Information', 'Smartphone Interface', 'Touch Screen Monitor', 'USB Port', 'Voice Recognition System', 'Remote Engine Start']

#print(Global_df)
print(len(Global_df.index))

for i  in  range(len(Global_df.index)) :    # row in Global_data frame
  ll = Global_df['Technology'].iloc[i]      # evaluate Technoloy list for row i
  ll = ast.literal_eval(ll)                 # cast it from string to list of strings
  print("2: i = row # ", i, "ll: Tech length: ", len(ll), "ll Tech list: ", ll)         
  for j  in  range(len(tech)) :           # each column (key) in sample_dict
    if  tech[j] in   ll :
      print("found1 column in Tech: ", tech[j], "\n")
      Global_df[tech[j]].iloc[i] = 1
      print("found2 column in Tech\n")
    else :
      print("Column is missing from Tech\n")
      Global_df[tech[j]].iloc[i]= 0

print("Global_df after initializing the expanded Technology cols: ", Global_df)

######################################################################
# Adam’s code to initialize the expanded Entertainment (7 columns) in 
# Global_df data frame.  Each row in Global_df will support a subset of
# the 7 columns for Entertainment (set to 1) and the remaining columns 
# (are set to 0)
#######################################################################


enter = ['iPod Connector', 'MP3 Player', 'Premium Radio', 'Satellite Radio', 'Bluetooth Streaming Audio', 'CD Player', 'iPod Interface']

for i  in  range(len(Global_df.index)) :    # row in Global_data frame
  ll = Global_df['Entertainment'].iloc[i]   # evaluate Entertainment list for row i
  ll = ast.literal_eval(ll)                 # cast it from string to list of strings
  print("2: i = row # ", i, "ll: Tech length: ", len(ll), "ll Tech list: ", ll)         
  for j  in  range(len(enter)) :           # each column (key) in sample_dict
    if  enter[j] in   ll :
      print("found1 column in Tech: ", enter[j], "\n")
      Global_df[enter[j]].iloc[i] = 1
      print("found2 column in Tech\n")
    else :
      print("Column is missing from Tech\n")
      Global_df[enter[j]].iloc[i]= 0

print("Global_df after initializing the expanded Entertainment cols: ", Global_df)


#print("5: row = 4, Entertainment: ", Global_df['Entertainment'].iloc[4])
#print("5: row = 4, Entertainment: ", Global_df['iPod Connector'].iloc[4], Global_df['MP3 Player'].iloc[4], Global_df['Premium Radio'].iloc[4], Global_df['Satellite Radio'].iloc[4], Global_df['Bluetooth Streaming Audio'].iloc[4], Global_df['CD Player'].iloc[4], Global_df['iPod Interface'].iloc[4] )
#print("6: row = 960, Technology: ", Global_df['Technology'].iloc[960])
#print("6: row = 960, Entertainment: ", Global_df['Entertainment'].iloc[960])

i = 707
print("7: row = i = ", i, "Technology: ", Global_df['Bluetooth Wireless Technology'].iloc[i], Global_df['Cruise Control'].iloc[i], Global_df['Hands Free Phone'].iloc[i], Global_df['Navigation System'].iloc[i], Global_df['Remote Keyless Entry'].iloc[i], Global_df['Audible Text Messaging'].iloc[i], Global_df['Internet Access'].iloc[i], Global_df['Proximity Sensing Keyless Entry'].iloc[i], Global_df['Push-Button Engine Start'].iloc[i], Global_df['Real-Time Traffic Information'].iloc[i], Global_df['Remote Engine Start'].iloc[i], Global_df['Smartphone Interface'].iloc[i], Global_df['USB Port'].iloc[i], Global_df['Voice Recognition System'].iloc[i], Global_df['Touch Screen Monitor'].iloc[i] )
print("7: row = i = ", i, "Entertainment: ", Global_df['iPod Connector'].iloc[i], Global_df['MP3 Player'].iloc[i], Global_df['Premium Radio'].iloc[i], Global_df['Satellite Radio'].iloc[i], Global_df['Bluetooth Streaming Audio'].iloc[i], Global_df['CD Player'].iloc[i], Global_df['iPod Interface'].iloc[i])

# Now, I have expanded Technology and Entertainment columns in the Global_df Data Frame
# We would like to generate CSV file for Global_df.csv to be stored on my local disk.
Global_df.to_csv (r'F:\AdamEzzat\Springboard\Makers-DFs\Global_df.csv')


In [ ]:
# Step-4: This file is to upload to Colab both Global_df.csv and Pakistan_df.csv files 
# that have the “Technology” and “Entertainment” columns expanded.
# Change the data type for Global_df data frame “Year” column to be “float” instead of “object”
# Merge the final version of Global_df and Pakistan and produce KBB_df final data frame based 
# on 3 columns (Maker/Brand, Model, Year). 

from     bs4 import  BeautifulSoup as soup
import   requests
import   pandas as pd

# Assume that we have Global_df and Pakistan_df CSV files
# Upload Global_df         # this step is done manually from Colab UI
Global_df = pd.read_csv(r'/content/Global_df.csv')

# Upload  Pakistan_df                          # this step is done manually from Colab UI
Pakistan_df = pd.read_csv(r'/content/Pakistan_df.csv' , encoding='cp1252')

# We have Global_df with expanded the Technology and Entertainment columns and is data 
# frame in Colab
# We have Pakistan_df as data frame in Colab


# Changing data type for column in a data frame 
# Global_df.info()    # prints the data types for all columns
Global_df['Year'] = Global_df.Year.astype (float)
Global_df.info()                                                                # print column name and data type

# Merge two data frames on common 3 columns (theta Join)
# Global_df.merge(Pakistan_df, left_on = ['Maker', 'Model','Year'], right_on = ['Brand', 'Model','Year'], how = 'inner')

KBB_df = Pakistan_df.merge (Global_df, left_on = ['Brand', 'Model','Year'], right_on = ['Maker', 'Model', 'Year'], how = 'inner') 

# download the KBB_df.csv to disk
from google.colab  import files
KBB_df.to_csv('KBB_df.csv')
files.download('KBB_df.csv')

